In [1]:
import pandas as pd
import numpy as np
import random
import os
import keras
from keras import models
from keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from scipy.ndimage import rotate
np.random.seed(42)

Using TensorFlow backend.


In [3]:
dd = pd.read_csv("train.csv")
dd.head()

M = int(0.8 * len(dd))

dd2 = dd[['x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']]
dd2 = dd2.sample(frac=1).reset_index(drop=True)

train_input = dd2.iloc[0:M][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].values
train_output = dd2.iloc[0:M][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values

test_input = dd2.iloc[M:][['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']].values
test_output = dd2.iloc[M:][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
# Acceso a fila
# dd.iloc[i]

# Accesso a columna
# dd['name']
# dd.name

In [4]:
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim = 6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal'))
    model.compile(loss = 'mean_squared_error', optimizer='adam')
    return model

In [6]:
model = create_model()
model.summary()
history = model.fit(train_input, train_output, epochs=5, batch_size=32)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 12)                84        
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 78        
Total params: 162
Trainable params: 162
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
519929/519929 [==============================] - 23s 44us/step - loss: 27291509.3113
Epoch 2/5
519929/519929 [==============================] - 22s 42us/step - loss: 22843004.7137
Epoch 3/5
519929/519929 [==============================] - 22s 42us/step - loss: 22825402.3613
Epoch 4/5
519929/519929 [==============================] - 22s 42us/step - loss: 22803226.3183
Epoch 5/5
519929/519929 [==============================] - 23s 44us/step - loss: 22793669.5159


In [9]:
results = model.evaluate(test_input, test_output)
results

129983/129983 [==============================] - 2s 18us/step


22837707.234107535